# Abstract

The film industry worldwide does upwards of 50 billion dollars in box office sales, not counting home entertainment revenue, which brings it up closer to 150 billion dollars. Operating within that brick of cash comes with a tremendous amount of risk, with films from major studios sometimes spending a quarter of a billion dollars or more on a single film. Decisions made at smaller studios are no less important to them, as they might be putting their entire livelihoods on the line in the hopes of a hit. And the value of a movie begins with a script.

My premise in creating this project was to ascertain whether a movie's critical rating can be determined to any extent by the text of its screenplay alone. Before attaching a cast and director and crew and all of the other costs associated with creating a cinematic work, having some guidance as to the quality of the script itself can be a benefit in minimizing risk. While an algorithm is no substitute for having a human eye on a screenplay, some level of unbiased machine learning can be leveraged to take a closer look. It is also possible to use this process to vet possible licensing if resources are short and slush piles are large.

As there are many factors going into a movie's rating, such as cast, director, editing, music, costuming, set design and so on, it is not necessarily expected that a movies critical rating can be determined solely by the text of its screenplay. However, there is still much value to be had if any measureable predictability can be found. At the completion of my modeling, I was ultimately able to predict scripts from good movies and bad movies, as rated by metacritic.com, about 65% of the time. Given the other factors in rating, I feel that is a significant enough to create value in the model. Among the various models I tried, some had a better true positive rate, and others had a better false positive rate, so there are some choices there depending on what use cases might be found, such as when it might be more advantageous to find a good movie versus avoiding a bad one.

Recommendations to a given sudio following would be to use modeling to sort potential screenplays into lists of scripts with higher likelihood of success, using modeling to evaluate scripts in process and step back to consider if it needs more work if the model doesn't like it, and for the screenwriters themselves, to check their scripts against the model and if it comes back with a 'bad' rating, potentially rethink their life choices.

For further research, I would like to create a text ingestion field in the dashboard that allows a user to insert a body of text and have a prediction returned evaluating the content as a screenplay and assigning a good or bad designation. I would create predictability functionality that would allow a user to choose a 'good' or 'bad' setting and have automatically generated text returned back in the style of either a good or bad screenplay. And I would go deeper into the neural networds when modeling, especially toward regression. They showed promise when using them in this project, but there was insufficient time to build them out using pre-made embedding layers and so on.

# Imports

In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import pandas as pd
import numpy as np
import importlib
import requests
import keras
import spacy
import time
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from mlxtend.plotting import plot_confusion_matrix
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from bs4 import BeautifulSoup
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm import tqdm
from nltk import word_tokenize
from ast import literal_eval
from importlib import reload

# Importing my own functions file.
import functions as fun

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
tqdm_notebook.pandas()

In [4]:
nlp = spacy.load('en_core_web_sm')

# Obtaining Data

All data for this report will be gathered using web scraping from the following web sites:
    
- metacritic.com for movie rating information.
- rottontomatoes.com for additional movie rating information.
- SpringfieldSpringfield.co.uk for gathering the screenplay texts.

## Rating Data

There were a few attempts at scraping data before finding versions that worked well for my purposes. What follows are the final attempts.

### Sraping metacritic.com

I'm taking the most highly rated and most lowly rated films as listed on this site. These extremes will be used for training my classification models to pridict if random movies will be highly rated or lowly rated films.

#### Great Movies

In [6]:
goods_titles = []

for i in range(0,20):
    # There are 10 pages to flip through of 100 movies each.
    page = requests.get(
        'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page={}'.format(i),
        headers={'User-Agent': 'Chrome/80.0.3987.116'})
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Now that we've gotten the content from the page, we need to loop through each element.
    for i in range(0,100,1):
        title = soup.find_all('span', class_="title numbered")[i]\
            .next_sibling.next_sibling.contents[1].contents[0]
        goods_titles.append(title)
    
    # We're only pinging 10 times but might as well be safe since it costs like
    # nothing.
    time.sleep(1)

Knowing that I won't be able to come up with screenplays for every single movie, I'm taking 2000 great and 2000 terrible films, in the hopes of winding up with at least 1000 of each.

In [ ]:
len(goods_titles)

In [ ]:
goods_titles[:5]

#### Terrible Movies

In [ ]:
bads_titles = []

for i in range(110, 130):
    # There are 10 pages to flip through of 100 movies each.
    page = requests.get(
        'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page={}'.format(i),
        headers={'User-Agent': 'Chrome/80.0.3987.116'})
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Now that we've gotten the content from the page, we need to loop through each element.
    for i in range(0,100,1):
        try:
            title = soup.find_all('span', class_="title numbered")[i]\
            .next_sibling.next_sibling.contents[1].contents[0]
            bads_titles.append(title)
        except:
            pass
    # We're only pinging 10 times but might as well be safe since it costs like
    # nothing.
    time.sleep(1)

In [ ]:
len(bads_titles)

In [ ]:
goods_formatted = fun.format_titles(goods_titles)
bads_formatted = fun.format_titles(bads_titles)

### Scraping rottentomatoes.com

The rottentomatoes.com information will be used for linear regression. Whereas with metacritic we were using only the best and worst for classification, here I'm using samples from the entire spectrum for regression analysis.

In [ ]:
all_rotten_movies = []
rotten_scores = []
for i in range(0, 101):
    page = requests.get("https://www.rottentomatoes.com/browse/"
                        "dvd-streaming-all?minTomato={}&maxTomato={}&services"
                        "=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;"
                        "fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14"
                        "&sortBy=release".format(i, i+1))
    soup = BeautifulSoup(page.content, 'html.parser')
    page = soup.get_text()
    comp = re.compile('"\/m\/\w+"')
    movies = comp.findall(page)
    movies_unique = list(set([movie[4:-1] for movie in movies]))
    rotten_scores.extend([i for _ in movies_unique])
    all_rotten_movies.extend(movies_unique)
    print(i)
    print(movies_unique)
    time.sleep(1)

In [ ]:
# Chopping off the year for those movies that have it.
rotten_movies_noyear = [film[:-5] if film[-4:-2] == '20' else film 
        for film in all_rotten_movies]

In [ ]:
rotten_form = fun.format_titles(rotten_movies_noyear)

## Scraping in the Screenplays

As this will be an analysis centered around natural language processing, my primary data source will be the screenplay text from every movie.

Unfortunately, I was eventually locked out of SpringfieldSpringfield.co.uk, the site where I retrieved the content from, due to too many 'visits'. As I had hit the site upwards of 10k times in the course of a few days, it was probably a fair call.

In [ ]:
# Getting the good screenplays along with a list of titles I couldn't find scripts for.
the_good, good_errors = fun.grab_screenplays(goods_formatted)

In [ ]:
len(good_errors)

In [ ]:
# Getting the good screenplays along with a list of titles I couldn't find scripts for.
the_bad, bad_errors = fun.grab_screenplays(bads_formatted)

In [ ]:
len(bad_errors)

Putting them both in a DataFrame to be used as data for the rest of the project.

In [ ]:
df_good = pd.DataFrame([the_good]).T
df_bad = pd.DataFrame([the_bad]).T

Now getting the screenplays for the rottentomatoes titles.

In [ ]:
rotten_movies, rotten_errors = fun.grab_screenplays(rotten_form)

In [ ]:
rotten_df = pd.DataFrame(columns=['titles',
                                  'titles_formatted',
                                  'rotten_scores',
                                  'scripts'])

In [ ]:
# Getting the titles and scores loaded into the DataFrame.
rotten_df.titles = rotten_form
rotten_df.RottenScores = rotten_scores

In [ ]:
# Getting things formatted correctly.
rotten_df.scripts = rotten_df.titles_formatted.apply(
    lambda x: rotten_scripts[0][x])

Loading all of this data into csv files to be saved in case of setbacks.

In [ ]:
df_good.to_csv('../project_resources/df_good_obtain.csv')
df_bad.to_csv('../project_resources/df_bad_obtain.csv')
rotten_df.to_csv('../project_resources/rotten_df_obtain.csv')

# Scrubbing Data

Reading back in data that was created in 'obtaining' file.

In [5]:
# I had intended to use files directly from the 'obtain' notebook to be cleaner,
# but I can't get back into the screenplay site, so I'll have to improvise.
# df_good = pd.read_csv('df_good_obtain.csv')
# df_bad = pd.read_csv('df_bad_obtain.csv')
# rotten_df = pd.read_csv('rotten_df_obtain.csv')

In [6]:
df_good = pd.read_csv('../project_resources/df_good.csv', index_col=False)
df_bad = pd.read_csv('../project_resources/df_bad.csv', index_col=False)
rotten_df = pd.read_csv('../project_resources/rotten_df.csv', index_col=0)

rotten_df.columns = ['titles', 'titles_formatted', 'rotten_scores', 
                     'scripts', 'all_together_now', 'no_stop', 'just_words']

## metacritic

### Setting up DataFrame

In [7]:
# Re-setting columns and index after re-importing.
df_good.columns = ['titles', 'scripts', 'good_or_bad']
df_bad.columns = ['titles', 'scripts', 'good_or_bad']

# Adding labels, combining good and bad, and dropping missing scripts.
df_good['good_or_bad'] = 1
df_bad['good_or_bad'] = 0

screenplays = pd.concat([df_good, df_bad])
screenplays.columns = ['titles', 'scripts', 'good_or_bad']

screenplays.dropna(inplace=True)

In [8]:
print('# of good screeenplays: ', len(screenplays[screenplays['good_or_bad'] == 1]))
print('# of bad screenplays: ', len(screenplays[screenplays['good_or_bad'] == 0]))

# of good screeenplays:  1270
# of bad screenplays:  1514


In [9]:
# converting imported screenplays back to lists from strings
screenplays.scripts = screenplays.scripts.apply(literal_eval)

# Series with a list of lines for each screenplay.
good_to_count = screenplays[screenplays['good_or_bad'] == 1]
bad_to_count = screenplays[screenplays['good_or_bad'] == 0]

# Single string of all good words.
splice_scripts = ''
for script in good_to_count['scripts']:
    splice_scripts += ''.join(script)

all_good_words = ''.join(splice_scripts)

# Single string of all bad words.
splice_scripts = ''
for script in bad_to_count['scripts']:
    splice_scripts += ''.join(script)

all_bad_words = ''.join(splice_scripts)

# Lists of all words lumped together and tokenized
good_data = word_tokenize(all_good_words)
bad_data = word_tokenize(all_bad_words)

### Script Metrics

Creating some simple word metrics.

In [10]:
print('good words total: ', len(good_data))
print('bad words total: ', len(bad_data))
print('----'*20)

print('good vocabulary: ', len(set(good_data)))
print('bad vocabulary: ', len(set(bad_data)))
print('----'*20)

print('good % vocab to total: ', round(len(set(good_data)) / len(good_data),4))
print('good % vocab to total: ', round(len(set(bad_data)) / len(bad_data),4))
print('----'*20)

# Total words divided by total number of sripts.
print('Average Good # Words: ', len(good_data) / len(good_to_count))
print('Average Bad # Words: ', len(bad_data) / len(bad_to_count))
print('----'*20)

print('Ave difference by words, good vs bad: ', 
      round(((len(good_data) / len(good_to_count)) \
       - (len(bad_data) / len(bad_to_count))) / (len(bad_data) \
                                                 / len(bad_to_count)),2))
print('----'*20)

good words total:  14020073
bad words total:  15477699
--------------------------------------------------------------------------------
good vocabulary:  172220
bad vocabulary:  183840
--------------------------------------------------------------------------------
good % vocab to total:  0.0123
good % vocab to total:  0.0119
--------------------------------------------------------------------------------
Average Good # Words:  11039.427559055119
Average Bad # Words:  10223.050858652576
--------------------------------------------------------------------------------
Ave difference by words, good vs bad:  0.08
--------------------------------------------------------------------------------


Counting punctuation and comparing good to bad.

In [11]:
for p in [':', ';', ',', '...', '!']:
    good_p = good_data.count(p) / len(good_data)
    bad_p = bad_data.count(p) / len(bad_data)
    print(f'Good \'{p}\' ratio: ', np.format_float_positional(round(good_p, 6)))
    print(f'Bad \'{p}\' ratio: ', np.format_float_positional(round(bad_p,6)))
    print(f'Good-Bad % for \'{p}\'', round((good_p - bad_p) / bad_p,2))
    print('-----'*10)

Good ':' ratio:  0.001625
Bad ':' ratio:  0.001428
Good-Bad % for ':' 0.14
--------------------------------------------------
Good ';' ratio:  0.000048
Bad ';' ratio:  0.000114
Good-Bad % for ';' -0.58
--------------------------------------------------
Good ',' ratio:  0.047181
Bad ',' ratio:  0.048443
Good-Bad % for ',' -0.03
--------------------------------------------------
Good '...' ratio:  0.011329
Bad '...' ratio:  0.010441
Good-Bad % for '...' 0.09
--------------------------------------------------
Good '!' ratio:  0.010268
Bad '!' ratio:  0.014268
Good-Bad % for '!' -0.28
--------------------------------------------------


Bad scripts on average use 28% more exclamation marks and 58% more semicolons.

Now creating additional columns for scripts in different formats. Right now, each script line is an element of a list. Breaking those apart into one long string.

In [12]:
splice_scripts = ''
for script in screenplays['scripts']:
    splice_scripts += ''.join(script)

all_words = ''.join(splice_scripts)

temp = []
for script in screenplays['scripts']:
    temp.append(''.join(script))

# This has each script as one long string inside of its cell, 
# as opposed with a list of lines.
screenplays['all_together_now'] = temp

data = word_tokenize(all_words)

Looking for # of unique tokens so I know roughly how many to play with when modeling.

In [13]:
len(set(data))

278462

The below function will be used in an apply function remove the stop words for purposes further down. One will keep punctuation and one will remove it.

In [14]:
screenplays['no_stop'] = screenplays['all_together_now']\
    .progress_apply(fun.stop_it, punct=False)

screenplays['just_words'] = screenplays['all_together_now']\
    .progress_apply(fun.stop_it, punct=True)

## Rottentomatoes

### Setting up DataFrame

In [15]:
rotten_df.dropna(inplace=True)

In [16]:
temp = []
for script in rotten_df.scripts:
    temp.append(''.join(script))

In [17]:
# This has each script as one long string inside of its cell, 
# as opposed with a list of lines.
rotten_df['all_together_now'] = temp

Using this function again for rotten_df this time to remove stop words for one columns of scripts and remove stop words plus punctuation for another.

In [18]:
rotten_df['no_stop'] = rotten_df['all_together_now']\
    .progress_apply(fun.stop_it, punct=False)

rotten_df['just_words'] = rotten_df['all_together_now']\
    .progress_apply(fun.stop_it, punct=True)

In [19]:
# Need to rename the period since it causes problems later on.
rotten_df.rename(columns={'.':'PER'}, inplace=True)

In [20]:
# rotten_df.dropna(inplace=True)
# screenplays.dropna(inplace=True)

rotten_df = rotten_df.drop_duplicates(subset=['titles']).copy()
screenplays = screenplays.drop_duplicates(['titles']).copy()

screenplays.to_csv('../project_resources/screenplays_scrub.csv')
rotten_df.to_csv('../project_resources/rotten_df_scrub.csv')

# Exploratory Data Analysis

## Local Functions

In [21]:
# Counting punctuation and parts of speech
def count_punct(text, punk):
    return text.count(punk) / len(text.split())

def count_pos(x):
    return x.count_by(spacy.attrs.POS)

def count_tag(x):
    return x.count_by(spacy.attrs.TAG)

# Determining the average sentence length for a given script.
def sent_len(str):
    doc = nlp(str)
    count = 0
    
    for sent in doc.sents:
        count += 1
    
    return len(str.split()) / count

# These two find the acutal POS name for the code number supplied, for
# creating the column names.
def POS_reverse_lookup_n_ratio(x, code):
    try:
        return x.POS_counts[code] / x.word_count
    except:
        pass

def TAG_reverse_lookup_n_ratio(x, code):
    try:
        return x.TAG_counts[code] / x.word_count
    except:
        return 0

## Bringing in the data

Reading back in data we scubbed in 'scrubbing'. 

In [22]:
screenplays = pd.read_csv('../project_resources/screenplays_scrub.csv', index_col=0)
rotten_df = pd.read_csv('../project_resources/rotten_df_scrub.csv', index_col=0)

screenplays.dropna(inplace=True)

In [23]:
# Adding column showing if a title is above or below a score of 50.
rotten_df['good_or_bad'] = rotten_df.rotten_scores.apply(
    lambda x: 1 if x >=50 else 0)

# Re-ordering columns to line up the two data sources for later use.
screen_cols = list(screenplays.columns)
screen_cols.remove('good_or_bad')
screen_cols.append('good_or_bad')
screenplays = screenplays[screen_cols].copy()

rotten_cols = list(rotten_df.columns)
rotten_cols.remove('rotten_scores')
rotten_cols.append('rotten_scores')
rotten_df = rotten_df[rotten_cols].copy()
rotten_df.drop('titles', axis=1, inplace=True)

In [24]:
# rotten_df = rotten_df[sort_cols]
rotten_df.rename(columns={'titles_formatted':'titles'}, inplace=True)

## Latent Derichlet Allocation

Finding the categories inherent in the screenplays by analyzing the words and grouping the screenplays.

In [25]:
combined_df = pd.concat([rotten_df[['titles', 'just_words']],
                         screenplays[['titles', 'just_words']]],
                         ignore_index=True)

combined_df.drop_duplicates(subset='titles', inplace=True)

In [26]:
cv = CountVectorizer(min_df=.1, max_df=.6)
victor = cv.fit_transform(combined_df.just_words)
LDA = LatentDirichletAllocation(n_components=10, random_state=42)
combined_df['category'] = LDA.fit_transform(victor).argmax(axis=1)

Showing the category word groupings, then creating materiels to be used later.

In [27]:
# This will house the dictionaries for using in the dashboard.
cat_word_dicts = []

# Priting out the word gropuings and filling in the cat_word_dicts list.
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR CATEGORY #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')
    cat_word_dicts.append({index:[cv.get_feature_names()[i]\
                                  for i in topic.argsort()[-15:]]})
#     cat_word_dicts.append([cv.get_feature_names()[i]\
#                           for i in topic.argsort()[-15:]])

cat_word_df = pd.DataFrame()

# Creating a dataframe for export to .csv for later use.
for i in range(len(cat_word_dicts)):
    cat_word_df[i] = cat_word_dicts[i][i]

# Creating category mapping for all dataframes to indicate how I see 
# The groupings. This part is just based on my looking them over.
category_map = {0: 'Dark & Political', 1: 'Sports, Comedy, Silly Horror', 
                2: 'Conflict', 3: 'Holiday, Films I Haven\'t Seen',
                4: 'Light-Hearted', 5: 'Unusual Language or Slang', 
                6: 'Violence & Gangster', 7: 'Romance & Light Drama', 
                8: 'Life Stories', 9: 'Straight Up Horror'}
combined_df['category_label'] = combined_df.category.map(category_map)

THE TOP 15 WORDS FOR CATEGORY #0
['government', 'fight', 'human', 'war', 'law', 'body', 'children', 'power', 'court', 'country', 'state', 'president', 'death', 'police', 'dr']


THE TOP 15 WORDS FOR CATEGORY #1
['buddy', 'wow', 'team', 'cool', 'uh', 'ball', 'aint', 'ya', 'ah', 'em', 'dog', 'ha', 'game', 'whoa', 'ok']


THE TOP 15 WORDS FOR CATEGORY #2
['air', 'safe', 'jim', 'ship', 'war', 'jack', 'officer', 'clear', 'shot', 'shoot', 'fire', 'police', 'john', 'captain', 'gun']


THE TOP 15 WORDS FOR CATEGORY #3
['ooh', 'whoa', 'ah', 'christmas', 'cool', 'wow', 'bye', 'daddy', 'charlie', 'sam', 'honey', 'jack', 'um', 'uh', 'mom']


THE TOP 15 WORDS FOR CATEGORY #4
['uh', 'grunts', 'radio', 'crowd', 'chatter', 'cheering', 'speaking', 'indistinct', 'continues', 'laughs', 'playing', 'laughing', 'sighs', 'chuckles', 'music']


THE TOP 15 WORDS FOR CATEGORY #5
['aint', 'poor', 'sister', 'darling', 'war', 'brother', 'child', 'death', 'lady', 'children', 'king', 'lord', 'dear', 'shall', 'mrs']


I'm going to need the combined dataframes with LDA categories for the dashboard later on.

In [28]:
screenplays = pd.merge(screenplays, combined_df[['titles','category', 'category_label']],
                how='left', on='titles')

rotten_df = pd.merge(rotten_df, combined_df[['titles','category', 'category_label']],
                how='left', on='titles')

cat_word_df.to_csv('../project_resources/cat_word_df.csv', header=True)

combined_df.to_csv('../project_resources/combined_df.csv')

## Metacritic

### Feature Engineering

Will be creating quite a few new columns filled with script attributes such as word counts, sentiment, and parts of speech.

Putting in some counts.

In [29]:
screenplays['word_count'] = screenplays.all_together_now.apply(
    lambda x: len(x.split()))
screenplays['unique_words'] = screenplays.no_stop.apply(
    lambda x: len(x.split()))

Creating columns for sentiment scoring: positive, negative, neutral, and compound.

In [30]:
sid = SentimentIntensityAnalyzer()

screenplays['sentiment_scores'] = screenplays.no_stop.progress_apply(
    lambda x: sid.polarity_scores(x))

screenplays['sentiment_negative'] = screenplays.sentiment_scores.apply(
    lambda x: x['neg'])
screenplays['sentiment_neutral'] = screenplays.sentiment_scores.apply(
    lambda x: x['neu'])
screenplays['sentiment_positive'] = screenplays.sentiment_scores.apply(
    lambda x: x['pos'])
screenplays['sentiment_compound'] = screenplays.sentiment_scores.apply(
    lambda x: x['compound'])

Removing anything with fewer words words so that short scripts don't throw off the modeling.

In [31]:
screenplays_cut = screenplays.copy().drop(
    index=(screenplays[screenplays['word_count'] < 1000].index), axis=0)

More columns for punctuation ratios.

In [32]:
screenplays_cut['colon_ratios'] = screenplays_cut.no_stop.apply(
    lambda x: count_punct(x, ':'))
screenplays_cut['semi_ratios'] = screenplays_cut.no_stop.apply(
    lambda x: count_punct(x, ';'))
screenplays_cut['comma_ratios'] = screenplays_cut.no_stop.apply(
    lambda x: count_punct(x, ','))
screenplays_cut['ellipsis_ratios'] = screenplays_cut.no_stop.apply(
    lambda x: count_punct(x, '...'))

And sentence length.

In [33]:
screenplays_cut['sentence_length'] = screenplays_cut.no_stop.progress_apply(
    sent_len)

### Parts of Speech

Breaking out both coarse and fine grained approaches to parts of speech, to create a ratio of how many of each is used in each screenplay.

In [34]:
screenplays_cut['nlp'] = screenplays_cut.all_together_now.progress_apply(nlp)
screenplays_cut.to_csv('../project_resources/screenplays_cut.csv')

In [35]:
# Applying the 'progress_apply' function to each row to get the actual words.
screenplays_cut['POS_counts'] = screenplays_cut.nlp.progress_apply(count_pos)
screenplays_cut['TAG_counts'] = screenplays_cut.nlp.progress_apply(count_tag)

# POS_codes and TAG codes will be used to identify unique codes 
# used throughout vocabulary.
POS_codes = set()

for i, item in enumerate(screenplays_cut.POS_counts):
    POS_codes.update(item.keys())

TAG_codes = set()

for i, item in enumerate(screenplays_cut.TAG_counts):
    TAG_codes.update(item.keys())

In [36]:
all_scripts = set()

for i, script in enumerate(screenplays_cut.just_words):
    all_scripts.update(script.split())

all_scripts = [script for script in all_scripts]

all_scripts = ' '.join(all_scripts)

all_scripts = nlp(all_scripts[:30])

Adding POS columns.

In [37]:
# TAG_lookup will be used for adding POS word to column name.
TAG_lookup = {}
for code in TAG_codes:
    key = code
    value = all_scripts.vocab[code].text
    TAG_lookup[key] = value

POS_lookup = {}
for code in POS_codes:
    key = code
    value = all_scripts.vocab[code].text
    POS_lookup[key] = value

# Adding POS column and adding POS name as column name.
for code, abb in POS_lookup.items():
    screenplays_cut[abb] = screenplays_cut.apply(
        POS_reverse_lookup_n_ratio, args=[code], axis=1)

# Adding TAG column and adding TAG name as column name.
for code, abb in TAG_lookup.items():
    screenplays_cut[abb] = screenplays_cut.apply(
        TAG_reverse_lookup_n_ratio, args=[code], axis=1)

In [38]:
screenplays_cut = screenplays_cut.fillna(0)

screenplays_cut.to_csv('../project_resources/screenplays_cut.csv')

## Rottentomatoes

### Feature Engineering

As with metacritic, will be creating quite a few new columns filled with script attributes such as word counts, sentiment, and parts of speech.

Putting in some counts.

In [39]:
rotten_df['word_count'] = rotten_df.all_together_now.apply(
    lambda x: len(x.split()))
rotten_df['unique_words'] = rotten_df.no_stop.apply(
    lambda x: len(x.split()))

In [40]:
rotten_df_cut = rotten_df.copy().drop(
    index=(rotten_df[rotten_df['word_count'] < 1000].index), axis=0)

Creating columns for sentiment scoring: positive, negative, neutral, and compound.

In [ ]:
sid = SentimentIntensityAnalyzer()

rotten_df_cut['sentiment_scores'] = rotten_df.no_stop.progress_apply(
    lambda x: sid.polarity_scores(x))

rotten_df_cut['sentiment_negative'] = rotten_df_cut.sentiment_scores.apply(
    lambda x: x['neg'])
rotten_df_cut['sentiment_neutral'] = rotten_df_cut.sentiment_scores.apply(
    lambda x: x['neu'])
rotten_df_cut['sentiment_positive'] = rotten_df_cut.sentiment_scores.apply(
    lambda x: x['pos'])
rotten_df_cut['sentiment_compound'] = rotten_df_cut.sentiment_scores.apply(
    lambda x: x['compound'])

Removing anything with fewer words words so that short scripts don't throw off the modeling.

More columns for punctuation ratios.

In [ ]:
rotten_df_cut['colon_ratios'] = rotten_df_cut.no_stop.apply(
    lambda x: count_punct(x, ':'))
rotten_df_cut['semi_ratios'] = rotten_df_cut.no_stop.apply(
    lambda x: count_punct(x, ';'))
rotten_df_cut['comma_ratios'] = rotten_df_cut.no_stop.apply(
    lambda x: count_punct(x, ','))
rotten_df_cut['ellipsis_ratios'] = rotten_df_cut.no_stop.apply(
    lambda x: count_punct(x, '...'))

And sentence length.

In [ ]:
rotten_df_cut['sentence_length'] = rotten_df_cut.no_stop.progress_apply(
    sent_len)

### Parts of Speech

Breaking out both coarse and fine grained approaches to parts of speech, to create a ratio of how many of each is used in each screenplay.

In [ ]:
rotten_df_cut['nlp'] = rotten_df_cut.all_together_now.progress_apply(nlp)

In [ ]:
rotten_df_cut.to_csv('../project_resources/rotten_df_cut.csv')

In [ ]:
# rotten_df_cut = pd.read_csv('../project_resources/rotten_df_cut.csv')

In [ ]:
# Applying the 'progress_apply' function to each row to get the actual words.
rotten_df_cut['POS_counts'] = rotten_df_cut.nlp.progress_apply(count_pos)

rotten_df_cut['TAG_counts'] = rotten_df_cut.nlp.progress_apply(count_tag)

# POS_codes and TAG codes will be used to identify unique codes 
# used throughout vocabulary.
POS_codes = set()

for i, item in enumerate(rotten_df_cut.POS_counts):
    POS_codes.update(item.keys())

TAG_codes = set()

for i, item in enumerate(rotten_df_cut.TAG_counts):
    TAG_codes.update(item.keys())

Adding POS columns.

In [ ]:
# TAG_lookup will be used for adding POS word to column name.
TAG_lookup = {}
for code in TAG_codes:
    key = code
    value = all_scripts.vocab[code].text
    TAG_lookup[key] = value
    
POS_lookup = {}
for code in POS_codes:
    key = code
    value = all_scripts.vocab[code].text
    POS_lookup[key] = value

# Adding POS column and adding POS name as column name.
for code, abb in POS_lookup.items():
    rotten_df_cut[abb] = rotten_df_cut.apply(
        POS_reverse_lookup_n_ratio, args=[code], axis=1)

# Adding TAG column and adding TAG name as column name.
for code, abb in TAG_lookup.items():
    rotten_df_cut[abb] = rotten_df_cut.apply(
        TAG_reverse_lookup_n_ratio, args=[code], axis=1)

In [ ]:
rotten_df_cut = rotten_df_cut.fillna(0)

In [ ]:
rotten_df_cut.to_csv('../project_resources/rotten_df_cut.csv')

Note that in the original eda.ipynb file, I have an extensive set of scatter matrices, histograms, and correlation matrices at this point in the process. However, they took up too much memory to be efficiently placed in this document, but they can be referred to in eda.ipynb as needed.

# Modeling

## Bring in the data

Data previously saved in .csv format from eda.ipynb.

In [ ]:
rotten_df_cut = pd.read_csv('../project_resources/rotten_df_cut.csv', index_col=0)
screenplays_cut = pd.read_csv('../project_resources/screenplays_cut.csv', index_col=0)

Shuffling columns  for ease of use when modeling.

In [ ]:
screen_cols = list(screenplays_cut.columns)
screen_cols.remove('good_or_bad')
screen_cols.append('good_or_bad')
screenplays_model = screenplays_cut[screen_cols].copy()

rotten_cols = list(rotten_df_cut.columns)
rotten_cols.remove('good_or_bad')
rotten_cols.append('good_or_bad')
rotten_cols.remove('rotten_scores')
rotten_cols.append('rotten_scores')
rotten_model = rotten_df_cut[rotten_cols].copy()

In [ ]:
# sort_cols = list(rotten_model.columns[:38])
# sort_cols.extend(sorted(list(rotten_model.columns[38:-2])))
# sort_cols.extend(list(rotten_model.columns[-2:]))

## TFIDF Vectorization

My first set of models will employ TFIDF Vectorization using a variety of classifiers.m

### Setup

For the modeling, I'm going to use the extreme ratings I got from metacritic.com for training the models, then use the rottentomatoes data, which is uniformly distributed from bad movies to good movies, to test the modeling.

In [ ]:
X_train = screenplays_cut.no_stop
X_test = rotten_df_cut.no_stop

y_train = screenplays_cut.good_or_bad
y_test = rotten_df_cut.good_or_bad

### Linear SVC

In [ ]:
fun.hybrid_classifiers(X_train, X_test, y_train, y_test, LinearSVC())

### SVC

In [ ]:
fun.hybrid_classifiers(X_train, X_test, y_train, y_test, SVC())

### XGBoost Classifier

In [ ]:
fun.hybrid_classifiers(X_train, X_test, y_train, y_test, 
                   XGBClassifier(max_depth=8,
                                    criterion='entropy',
                                    min_samples_split=14,
                                    min_samples_leaf=1,
                                    max_features=160))

### Neural Network

Because the neural network works so differently from the other classifiers, I haven't used a function, and the main code is all here.

Starting to put the data together.

In [ ]:
X_train = pd.DataFrame(X_train)

# Undersampling to get things even just for ease of use.
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(pd.DataFrame(X_train), y_train)

X_resampled = X_resampled.iloc[:, 0]

Create the vector from the full batch of scripts.

In [ ]:
tfidf = TfidfVectorizer(max_features=5000, max_df=.95, min_df=.1, 
                        ngram_range=(1,2))
X_t_train = tfidf.fit_transform(X_resampled)
X_t_test = tfidf.transform(X_test)

Getting the data into the format it will need to be in for the network.

In [ ]:
X_t_train = pd.SparseDataFrame(X_t_train, columns=tfidf.get_feature_names(),
                           default_fill_value=0)
X_t_test = pd.SparseDataFrame(X_t_test, columns=tfidf.get_feature_names(),
                           default_fill_value=0)

X_t_num = np.array(X_t_train)
X_t_test_num = np.array(X_t_test)

y_t_num = np.array(y_resampled)
y_t_test_num = np.array(y_test)

layer_input = X_t_train.shape[1]

I played manually with a lot of different layers, neurons, drop layers, and regularization. Simple seemed to work best.

In [ ]:
model = Sequential()

model.add(Dense(layer_input, input_dim=layer_input, activation='relu'))
model.add(Dense(50, input_dim=layer_input, activation='relu'))
model.add(Dense(50, input_dim=50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='mean_squared_error',
              optimizer='adam', 
              metrics=['accuracy'])

model.fit(X_t_num, y_t_num, epochs=4, batch_size=50, validation_split=0.2)

In [ ]:
# y_t_test = y_test

In [ ]:
model.evaluate(X_t_test_num, y_t_test_num, verbose=1)

In [ ]:
y_pred = model.predict_classes(X_t_test_num)

print(classification_report(y_test, y_pred))

confusion = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(confusion, figsize=(7,7), colorbar=True,
                  show_normed=True, cmap=plt.cm.Reds);
plt.show();

Overall, a pretty positive model.

### TFIDF Summary

The models here were strong, overall. Most of them had accuracy of around 65%, with poor true positive rate of around 50, which means they were quite good at predicting which scripts were bad but not great at predicting which scripts were good. The decision tree had good true positive rate at 61%, but the accuracy wasn't as good at 58%. XG Boost had the best accuracy at 66, but the true positive rate of precisely 50%, with true negative rate 80%.

## Script Attributes

### Setup

For this set of modeling, I'm using only the script attributes from the dataframe, so basically word counts, punctuation ratios, sentence length, sentiment, and parts of speech.

In [ ]:
screen_dummies = pd.get_dummies(screenplays_model['category'])
rotten_dummies = pd.get_dummies(rotten_model['category'])

In [ ]:
columns = list(screenplays_model.columns[7:9])
columns.extend(list(screenplays_model.columns[10:19]))
start = list(screenplays_model.columns).index('PROPN')
columns.extend(list(screenplays_model.columns[start:-1]))
    
X = screenplays_model[columns]
X_train = pd.merge(screenplays_model[columns], screen_dummies, left_index=True, 
             right_index=True)
X_test = pd.merge(rotten_model[columns], rotten_dummies, left_index=True, 
             right_index=True)

y_train = screenplays_model.good_or_bad
y_test = rotten_model.good_or_bad

### XG Boost Classifier

In [ ]:
fun.hybrid_classifiers(X_train, X_test, y_train, y_test, 
                   classifier=XGBClassifier(),use_tfidf=False)

### Random Forest Classifier

In [ ]:
fun.hybrid_classifiers(X_train, X_test, y_train, y_test, 
                   classifier=RandomForestClassifier(random_state=42,
                                                    max_depth=8,
                                                    criterion='entropy',
                                                    min_samples_split=14,
                                                    min_samples_leaf=1,
#                                                     max_features=10),
                                                    ),
                  use_tfidf=False)

### Scripts Attributes Summary

This set was much lower than the TFIDF, with scores that were only a little above random chance. The neural network simply classified everything as positive. I'm sure I could have done something to improve this batch given time, but for now, as it was so much lower than the TFIDF set, I chose to let it go.

## Combined TDIF & Attributes

### Setup

For this more experimental bunch, I thought I would try merging the TFIDF vecctorization matrix with the attributes from the original dataframe, wondering if more information might be better, and how the engineered features might play against the word matrix.

In [ ]:
screen_dummies = pd.get_dummies(screenplays_model['category'])
rotten_dummies = pd.get_dummies(rotten_model['category'])

In [ ]:
columns = list(screenplays_model.columns[7:9])
columns.extend(list(screenplays_model.columns[10:19]))
start = list(screenplays_model.columns).index('PROPN')
columns.extend(list(screenplays_model.columns[start:-1]))

X_train = screenplays_model[columns].copy()
X_test = rotten_model[columns].copy()

y_train = screenplays_model.good_or_bad.copy()
y_test = rotten_model.good_or_bad.copy()

In [ ]:
X2_train = screenplays_model.no_stop
X2_test = rotten_model.no_stop
y_train = screenplays_model.good_or_bad
y_test = rotten_model.good_or_bad

### Linear Support Vector Classifier

In [ ]:
# temp = pd.DataFrame(X2_train, columns=['temp'])
# temp[temp['temp'].isna() == True].index

In [ ]:
# screenplays_model.iloc[[40, 69, 101, 106, 147, 175, 264, 303, 343, 371, 392,
#              464, 656, 811, 963, 1099, 2024, 2044, 2066, 2265, 2554, 2600]]

In [ ]:
fun.hybrid_classifier_combo(X_train, X_test, X2_train, X2_test,
                        y_train, y_test, LinearSVC(C=.6))

### Logistic Regression Classifier

In [ ]:
fun.hybrid_classifier_combo(X_train, X_test, X2_train, X2_test,
                        y_train, y_test, LogisticRegression(C=10))

### Random Forest Classifier

In [ ]:
fun.hybrid_classifier_combo(X_train, X_test, X2_train, X2_test,
                        y_train, y_test, 
                        RandomForestClassifier(random_state=42, 
                                               n_jobs=-1),
                        feature_importance=False)

## Stacked Modeling

This is one of my top models from the TFIDF modeling section. I'll use it as the first phase of a two-part modeling scenario.

First, I'm going to send in the movies from metacritic to train the first model, using the rottentomatoes data for testing. Then I'll marry the test predictions to the rottentomatoes (test) data, and send it through the second model, where it will get split into a proper train-test split.

In [ ]:
X_train = screenplays_cut.no_stop
X_test = rotten_df_cut.no_stop

y_train = screenplays_cut.good_or_bad
y_test = rotten_df_cut.good_or_bad

In [ ]:
model_1_predictions = fun.hybrid_classifiers(X_train, X_test, y_train, y_test, SVC())

In [ ]:
# X = screenplays_model.no_stop
# y = screenplays_model.good_or_bad

# model_1_predictions = fun.stacked_classifier(X, y, SVC())

Below I'm getting the dummies and putting them all in one dataframe to be joined later.

In [ ]:
screen_dummies = pd.get_dummies(screenplays_model['category'])
rotten_dummies = pd.get_dummies(rotten_model['category'])

In [ ]:
# columns = ['sentiment_neutral', 'sentence_length', 'PRON', 'CCONJ', 'PUNCT', 'NNS',
#  '_SP', 'VBD', 'WDT', 'VB', 'PRP', 'RP', 'PRP$', 'CC', '.', 'IN', '-RRB-',
#  'VBP', 'WP', 'HYPH']

columns = ['word_count', 'unique_words']
columns.extend(rotten_model.columns[10:19])
columns.extend(rotten_model.columns[23:-2])

X = pd.merge(rotten_model[columns].copy(), rotten_dummies, left_index=True, 
             right_index=True)

y = rotten_model.good_or_bad.copy()

In [ ]:
X['predictions'] = model_1_predictions

Using the test data from previously, since it had not yet seen the overfit training model.

In [ ]:
fun.script_classifiers(X, y,
                   classifier=RandomForestClassifier(random_state=42,
                                                    max_depth=9,
                                                    criterion='entropy',
                                                    min_samples_split=14,
                                                    min_samples_leaf=1,
                                                    n_estimators=103),
                      use_tfidf=False, test_size=.2)

This turned out to be my best model: SVC using TFIDF, then take the predictions from that, add them as a feature (to test data that has not been modeled) and run that data again through a random forest classifier: 68% accuracy, with a true positive rate of 64% and a true negative rate of 72%. Besides having the greatest accuracy, this model also turned out to be the most balanced.

## Linear Regression with Rottentomatoes

Next I attempted some regression models using data from rottontomatoes.

The models here are only the ones I kept. The others had results that were basically no better than chance.

### TFIDF with XGBoost Regressor

I grid searched the hell out of this one, only to find that none of it mattered. So I removed the grid search function.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rotten_model.no_stop,
                                                    rotten_model.rotten_scores,
                                                    test_size=.3,
                                                    random_state=42)

tfidf = TfidfVectorizer(max_features=5000, max_df=.9, min_df=.1, 
                        ngram_range=(1,2))
word_predictors = tfidf.fit_transform(X_train)
word_test = tfidf.transform(X_test)

model = XGBRegressor(random_state=42, n_estimators=100, 
                     objective='reg:squarederror')
model.fit(word_predictors, y_train)

r^2 and mse for train and test.

In [ ]:
def get_regression_scores(X_train, X_test, y_train_y_test, model):
    y_hat_train = model.predict(word_predictors)
    y_hat_test = model.predict(word_test)
    print('train MSE score: ', mse(y_train, y_hat_train))
    print('train r2_score: ', r2_score(y_train, y_hat_train))
    print('Test MSE score:', mse(y_test, y_hat_test))
    print('Test R-sq score:', r2_score(y_test, y_hat_test))

In [ ]:
y_hat_train = model.predict(word_predictors)
y_hat_test = model.predict(word_test)
print('train MSE score: ', mse(y_train, y_hat_train))
print('train r2_score: ', r2_score(y_train, y_hat_train))
print('Test MSE score:', mse(y_test, y_hat_test))
print('Test R-sq score:', r2_score(y_test, y_hat_test))

In [ ]:
train_residuals = y_hat_train - list(y_train)
test_residuals = y_hat_test - list(y_test)
print('Ave deviation from actual: ', round(sum(abs(test_residuals)) / len(test_residuals),2))

Creating a dataframe comparing predicted scores to actual scores.

In [ ]:
actual_v_predicted = X_test.to_frame()

pred_scores = list(y_hat_test)

actual_v_predicted['predicted_scores'] = pred_scores

to_merge = rotten_model[['titles', 'rotten_scores']]

actual_v_predicted = actual_v_predicted.merge(to_merge, left_index=True,
                                              right_index=True)

actual_v_predicted = actual_v_predicted[['titles','rotten_scores',
                                         'predicted_scores']]

actual_v_predicted['predicted_scores'] = actual_v_predicted.predicted_scores.\
    apply(lambda x: int(x))

In [ ]:
actual_v_predicted

In [ ]:
upside = len(actual_v_predicted[(actual_v_predicted.rotten_scores > 50) \
                   & (actual_v_predicted.predicted_scores > 50)])
upside

downside = len(actual_v_predicted[(actual_v_predicted.rotten_scores < 50) \
                   & (actual_v_predicted.predicted_scores < 50)])
downside

all_sides = len(actual_v_predicted)
all_sides

capture_in_half = (upside + downside) / len(actual_v_predicted)
print('Scores captured in same half (upper vs lower): ', capture_in_half)

In [ ]:
actual_v_predicted.rotten_scores.hist();

In [ ]:
actual_v_predicted.predicted_scores.hist();

In [ ]:
fig = sm.graphics.qqplot(test_residuals, dist=stats.norm, line='45', fit=True)
fig.show();

In [ ]:
actual_v_predicted

### Features Only with XG Boost Regressor

Setting up all the features for a features-only (non-tfidf) regression model.

In [ ]:
# Need to change this as the model chokes on the period.
rotten_model.rename(columns={'.':'PER'}, inplace=True)

# Dropping out the unnecessary columns.
to_drop = list(rotten_model.columns[:7])
to_drop.append(rotten_model.columns[9])
to_drop.extend(rotten_model.columns[19:22])
to_drop.extend(rotten_model.columns[-2:])

to_drop_2 = ['sentiment_compound', 'unique_words', 'X', 'JJ', 'NNP', 'DET',
             'semi_ratios', 'ADJ']

predictors_all = rotten_df_cut.copy()
predictors_all.drop(to_drop, axis=1, inplace=True)
predictors_all.drop(to_drop_2, axis=1,inplace=True)

# Adding dummy categories.
# rotten_dummies = pd.get_dummies(rotten_model['category'])
# predictors_all = pd.merge(predictors_all, rotten_dummies, left_index=True, 
#              right_index=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(predictors_all,
                                                    rotten_model.rotten_scores,
                                                    test_size=.3)

In [ ]:
scaler = StandardScaler()
scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train), 
                          columns = X_train.columns)
scaled_X_test = pd.DataFrame(scaler.transform(X_test), 
                            columns=X_test.columns)

In [ ]:
scaled_y_train = scaler.fit_transform(np.array(y_train).reshape(-1,1))
scaled_y_test = scaler.transform(np.array(y_test).reshape(-1,1))
scaled_y_train = pd.Series(scaled_y_train.reshape(-1,), name='rotten_scores')
scaled_y_test = pd.Series(scaled_y_test.reshape(-1,), name='rotten_scores')

In [ ]:
scaled_X_train.fillna(0, inplace=True)
scaled_X_test.fillna(0, inplace=True)

In [ ]:
def test(x):
    if x == 0:
        return math.log((X + 1) / 100)
    else:
        return math.log(x / 100)

In [ ]:
model = XGBRegressor(random_state=42, n_estimators=100, 
                     objective='reg:squarederror')
model.fit(scaled_X_train, y_train)

Making predictions and calculating residuals

In [ ]:
y_hat_train = model.predict(pd.DataFrame(scaled_X_train))
y_hat_test = model.predict(pd.DataFrame(scaled_X_test))

train_residuals = y_hat_train - list(y_train)
test_residuals = y_hat_test - list(y_test)

In [ ]:
print('Ave deviation from actual: ', round(sum(abs(test_residuals)) / len(test_residuals),2))

In [ ]:
mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test = np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

In [ ]:
print('R^2 score:', r2_score(y_test,y_hat_test))

In [ ]:
y_pred = model.predict(scaled_X_test)

Creating dataframe of predicted scores vs. actual

In [ ]:
actual_v_predicted = X_test.copy()

pred_scores = list(y_pred)

actual_v_predicted['predicted_scores'] = pred_scores

to_merge = rotten_model[['titles', 'rotten_scores']]

actual_v_predicted = actual_v_predicted.merge(to_merge, left_index=True,
                                              right_index=True)

actual_v_predicted = actual_v_predicted[['titles','rotten_scores',
                                         'predicted_scores']]

actual_v_predicted['predicted_scores'] = actual_v_predicted.predicted_scores.\
    apply(lambda x: int(x))

actual_v_predicted = actual_v_predicted[actual_v_predicted['predicted_scores'] <= 100]

In [ ]:
actual_v_predicted.predicted_scores.hist()

In [ ]:
actual_v_predicted

In [ ]:
fig = sm.graphics.qqplot(test_residuals, dist=stats.norm, line='45', fit=True)
fig.show()

### TFIDF with Features with XG Boost Regressor

In [ ]:
X = predictors_all

In [ ]:
tfidf = TfidfVectorizer(max_features=5000, max_df=.9, min_df=.1, 
                        ngram_range=(1,2))
X2 = tfidf.fit_transform(rotten_model.no_stop)

In [ ]:
# Creating a sparse DataFrame to house both the features and the 
# processed text.
X_temp = pd.SparseDataFrame(X2, columns=tfidf.get_feature_names(),
                           default_fill_value=0)
# Necessary for next step.
X = X.reset_index(drop=True)

# Combining text matrix with script attributes.
for column in X:
    X_temp[column] = X[column]
X = X_temp

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, random_state=42,
                                                    test_size=.3)

In [ ]:
scaler = StandardScaler()
scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train), 
                          columns = X_train.columns)
scaled_X_test = pd.DataFrame(scaler.transform(X_test), 
                            columns=X_test.columns)

In [ ]:
scaled_y_train = scaler.fit_transform(np.array(y_train).reshape(-1,1))
scaled_y_test = scaler.transform(np.array(y_test).reshape(-1,1))
scaled_y_train = pd.Series(scaled_y_train.reshape(-1,), name='rotten_scores')
scaled_y_test = pd.Series(scaled_y_test.reshape(-1,), name='rotten_scores')

In [ ]:
scaled_X_train.fillna(0, inplace=True)
scaled_X_test.fillna(0, inplace=True)

In [ ]:
def test(x):
    if x == 0:
        return math.log((X + 1) / 100)
    else:
        return math.log(x / 100)

In [ ]:
model = XGBRegressor(random_state=42, n_estimators=100, 
                     objective='reg:squarederror')
model.fit(scaled_X_train, y_train)

In [ ]:
y_hat_train = model.predict(pd.DataFrame(scaled_X_train))
y_hat_test = model.predict(pd.DataFrame(scaled_X_test))

In [ ]:
train_residuals = y_hat_train - list(y_train)
test_residuals = y_hat_test - list(y_test)

In [ ]:
mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test = np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

In [ ]:
print('r^2 score:', r2_score(y_test,y_hat_test))

Overall, the regression didn't work out that well. I probably could have done more with it with more time, and it would be interesting to get deeper into it. 